# Generating Combined Dataset for Analysis
---
This notebook was originally managed and run in Google Colaboratory. For the purposes of this notebook, we have commented out the Colab paths, but have kept them for future reference. The resulting dataset from this notebook will contain water, temperature, drought, and income data in snake-case form, and with null values appropriately processed.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Imports

In [ ]:
import pandas as pd
import numpy as np


#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reading CSV files from google drive.  File paths should work for you if the project folder is top level, otherwise please adjust as needed

In [ ]:
#drought = pd.read_csv('/content/drive/MyDrive/group_project_data/clean_data/annual_drought_2015_2020.FIPS.csv')
#temp = pd.read_csv('/content/drive/MyDrive/group_project_data/clean_data/annual_temperature_2000-2020_FIPS.csv')
#counties = pd.read_csv('/content/drive/MyDrive/group_project_data/clean_data/counties.csv')
#income = pd.read_csv('/content/drive/MyDrive/group_project_data/clean_data/county_median_income_2015.csv')
#usco = pd.read_csv('/content/drive/MyDrive/group_project_data/clean_data/usco2015v2.0_clean2.csv')

drought = pd.read_csv('../../data/raw-data/annual_drought_2000_2020.FIPS.csv')
temp = pd.read_csv('../../data/raw-data/clean_data/annual_temperature_2000-2020_FIPS.csv')
counties = pd.read_csv('../../data/raw-data/clean_data/counties.csv')
income = pd.read_csv('../../data/raw-data/county_median_income_2015.csv')
usco = pd.read_csv('../../data/raw-data/usco2015v2.0_clean2.csv')

## Adjusting data sets before merging

In [ ]:

#filtering for just year 2015
drought2015 = drought[drought['year'] ==2015]
#droping index column and year
drought2015.drop(columns = ['year'], inplace = True)
drought2015.head()

<ipython-input-194-461aa66b92fb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drought2015.drop(columns = ['year'], inplace = True)


,FIPS,normal_wet,abnormally_dry,moderate_drought,severe_drought,extreme_drought,exceptional_drought
48315,10001,98.35,1.65,0.00,0.00,0.0,0.0
48316,10003,99.54,0.46,0.00,0.00,0.0,0.0
48317,10005,94.59,5.41,0.00,0.00,0.0,0.0
48318,1001,54.11,45.89,0.00,0.00,0.0,0.0
48319,1003,39.10,60.90,28.14,0.97,0.0,0.0


In [ ]:
#filtering for just year 2015
temp2015 = temp.drop(temp[temp['Year'] != 2015].index, inplace=True)

#renaming stcoFIPS to FIPS to help with meerge
temp2015 =temp.rename(columns={'StCoFIPS': 'FIPS'})

#dropping year
temp2015.drop(columns = ['Year'], inplace = True)
temp2015.head()

,FIPS,Tmean_C,TDmean_C,NETmean_C,HImean_C,HXmean_C,WBGTmean_C,Flag_T,Flag_TD,Flag_NET,Flag_HI,Flag_HX,Flag_WBGT
46620,1001,19.05,13.45,13.25,19.41,23.06,17.78,0.0,0.0,0.0,0.0,0.0,0.0
46621,1003,20.84,16.15,14.68,21.67,26.45,19.83,0.0,0.0,0.0,0.0,0.0,0.0
46622,1005,19.54,14.19,14.14,19.94,23.93,18.39,0.0,0.0,0.0,0.0,0.0,0.0
46623,1007,18.45,12.91,12.99,18.76,22.26,17.33,0.0,0.0,0.0,0.0,0.0,0.0
46624,1009,17.18,11.75,11.52,17.31,20.38,16.10,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#removing unnamed: 0
counties.drop(columns = ['Unnamed: 0'], inplace = True)
counties.head()


,STATE,COUNTYNAME,FIPS,LON,LAT
0,ME,Washington,23029,-67.6361,45.0363
1,GA,McIntosh,13191,-81.2646,31.5329
2,GA,Liberty,13179,-81.2103,31.7093
3,AS,Swains Island,60040,-171.0459,-11.0843
4,AS,Manu'a,60020,-169.5060,-14.2219


In [ ]:
#removing county column so we don't have duplicates after the merge
income.drop(columns = ['County', 'State Code', 'State', 'Population'], inplace = True)
income.head()

,Median household income,FIPS
0,51281.0,1001.0
1,50254.0,1003.0
2,32964.0,1005.0
3,38678.0,1007.0
4,45813.0,1009.0


In [ ]:
#removing county, state, and year (all 2015) column so we don't have duplicates after the merge
usco.drop(columns = ['COUNTY', 'STATE', 'YEAR'], inplace = True)
usco.head()

,FIPS,TP-TotPop,PS-GWPop,PS-SWPop,PS-TOPop,PS-WGWTo,PS-WSWTo,PS-Wtotl,DO-SSPop,DO-SSPCp,DO-PSDel,DO-PSPCp,DO-WFrTo,DO-WDelv,IN-WGWTo,IN-WSWTo,IN-Wtotl,IR-WFrTo,IR-RecWW,IR-CUsFr,IR-IrTot,IC-WFrTo,IC-RecWW,IC-CUsFr,IC-IrTot,IG-WFrTo,IG-RecWW,IG-CUsFr,IG-IrTot,LI-WFrTo,AQ-WGWTo,AQ-WSWTo,AQ-Wtotl,MI-WGWTo,MI-WSWTo,MI-Wtotl,PT-WGWTo,PT-WSWTo,PT-Wtotl,PT-RecWW,PT-PSDel,PT-CUTot,PT-Power,PO-WGWTo,PO-WSWTo,PO-Wtotl,PO-RecWW,PO-PSDel,PO-CUTot,PO-Power,PC-WGWTo,PC-WSWTo,PC-Wtotl,PC-RecWW,PC-PSDel,PC-CUTot,PC-Power,TO-WGWTo,TO-WSWTo,TO-Wtotl,TO-CUTotPartial
0,1001,55.347,--,--,48.998,3.64,0.00,3.64,6.349,60.0,2.71,55.0,0.38,3.09,1.83,32.97,34.8,3.52,--,3.52,2.26,3.36,--,3.36,2.16,0.16,--,0.16,0.10,0.19,0.00,0.00,0.00,1.40,0.62,2.02,0.0,11.86,11.86,--,--,9.66,15711.64,0.0,0.0,0.0,--,--,0.0,0.0,0.0,11.86,11.86,--,--,9.66,15711.64,10.69,45.72,56.41,13.18
1,1003,203.709,--,--,174.907,23.67,0.00,23.67,28.802,74.0,12.78,73.0,2.14,14.92,0.00,0.00,0.0,58.28,--,58.28,20.66,53.95,--,53.95,18.52,4.33,--,4.33,2.14,0.29,0.05,0.05,0.10,0.22,0.00,0.22,0.0,0.00,0.00,--,--,0.00,0.00,0.0,0.0,0.0,--,--,0.0,0.0,0.0,0.00,0.00,--,--,0.00,0.00,73.38,11.32,84.70,58.28
2,1005,26.489,--,--,24.587,3.23,0.00,3.23,1.902,63.0,1.42,58.0,0.12,1.54,1.50,0.00,1.5,2.96,--,2.96,4.53,2.49,--,2.49,4.23,0.47,--,0.47,0.30,0.41,0.18,0.18,0.36,0.30,0.14,0.44,0.0,0.00,0.00,--,--,0.00,0.00,0.0,0.0,0.0,--,--,0.0,0.0,0.0,0.00,0.00,--,--,0.00,0.00,6.06,2.96,9.02,2.96
3,1007,22.583,--,--,20.947,5.18,0.00,5.18,1.636,98.0,1.81,86.0,0.16,1.97,0.00,0.00,0.0,0.19,--,0.19,0.16,0.03,--,0.03,0.06,0.16,--,0.16,0.10,0.07,0.00,0.00,0.00,0.27,0.00,0.27,0.0,0.00,0.00,--,--,0.00,0.00,0.0,0.0,0.0,--,--,0.0,0.0,0.0,0.00,0.00,--,--,0.00,0.00,5.67,0.20,5.87,0.19
4,1009,57.673,--,--,44.815,2.78,54.08,56.86,12.858,67.0,2.79,62.0,0.86,3.65,0.00,0.00,0.0,1.08,--,1.08,0.94,0.35,--,0.35,0.53,0.73,--,0.73,0.41,0.90,0.00,0.00,0.00,0.37,0.00,0.37,0.0,0.00,0.00,--,--,0.00,0.00,0.0,0.0,0.0,--,--,0.0,0.0,0.0,0.00,0.00,--,--,0.00,0.00,4.64,55.43,60.07,1.08


# Merging all Data Together

In [ ]:
df = pd.merge(counties, usco, on='FIPS', how = 'left')
df = pd.merge(df, income, on='FIPS', how = 'left')
df = pd.merge(df, temp2015, on='FIPS', how = 'left')
df = pd.merge(df, drought2015, on='FIPS', how = 'left')

# Changing column names to snake case

In [ ]:
df = df.rename(columns=lambda x: x.lower().replace('-', '_').replace(' ', '_'))


# Removing rows that are NAN 98% of the time (those that have only the FIPS, STate, lon and lat values)

In [ ]:
#making a list of the 55 counties that we do not have data on first.
na_counties = df[df['tp_totpop'].isnull()]
na_counties = na_counties.iloc[:, :5]
#na_counties.to_csv('/content/drive/MyDrive/group_project_data/clean_data/na_counties.csv', index=False)
na_counties.to_csv('../../data/raw-data/na_counties.csv', index=False)

In [ ]:
#dropping 98% NAN
df = df.dropna(thresh=int(0.02 * len(df)))


# Hawaii and Alaska and PR have a lot of missing data, so we will remove those two states as we are not able to work with them and they are so different climate wise that they could be considered outlyers


In [ ]:
df = df[~df['state'].isin(['AK', 'HI', 'PR'])]

#Converting NAN to 0 for columns that are day or supply based.  

In [ ]:
df['normal_wet'] = df['normal_wet'].fillna(0)
df['moderate_drought'] = df['moderate_drought'].fillna(0)
df['severe_drought'] = df['severe_drought'].fillna(0)
df['extreme_drought'] = df['extreme_drought'].fillna(0)
df['exceptional_drought'] = df['exceptional_drought'].fillna(0)
df['abnormally_dry'] = df['abnormally_dry'].fillna(0)
df['do_sspcp'] = df['do_sspcp'].fillna(0)
df['do_pspcp'] = df['do_pspcp'].fillna(0)

## Replacing NAN population and Temp with the mean of their state

In [ ]:
columns_to_update = ['median_household_income', 'tmean_c', 'tdmean_c', 'netmean_c', 'himean_c',
                     'hxmean_c', 'wbgtmean_c', 'flag_t', 'flag_td', 'flag_net', 'flag_hi', 'flag_hx', 'flag_wbgt']
state_group = df.groupby('state')
state_means = state_group[columns_to_update].transform('mean')
for column in columns_to_update:
  df[column] = df[column].fillna(state_means[column])

In [ ]:
df.head()

,state,countyname,fips,lon,lat,tp_totpop,ps_gwpop,ps_swpop,ps_topop,ps_wgwto,ps_wswto,ps_wtotl,do_sspop,do_sspcp,do_psdel,do_pspcp,do_wfrto,do_wdelv_,in_wgwto,in_wswto,in_wtotl,ir_wfrto,ir_recww,ir_cusfr,ir_irtot,ic_wfrto,ic_recww,ic_cusfr,ic_irtot,ig_wfrto,ig_recww,ig_cusfr,ig_irtot,li_wfrto,aq_wgwto,aq_wswto,aq_wtotl,mi_wgwto,mi_wswto,mi_wtotl,pt_wgwto,pt_wswto,pt_wtotl,pt_recww,pt_psdel,pt_cutot,pt_power,po_wgwto,po_wswto,po_wtotl,po_recww,po_psdel,po_cutot,po_power,pc_wgwto,pc_wswto,pc_wtotl,pc_recww,pc_psdel,pc_cutot,pc_power,to_wgwto,to_wswto,to_wtotl,to_cutotpartial,median_household_income,tmean_c,tdmean_c,netmean_c,himean_c,hxmean_c,wbgtmean_c,flag_t,flag_td,flag_net,flag_hi,flag_hx,flag_wbgt,normal_wet,abnormally_dry,moderate_drought,severe_drought,extreme_drought,exceptional_drought
0,ME,Washington,23029,-67.6361,45.0363,31.625,9.446,1.900,11.346,1.38,0.19,1.57,20.279,53.0,0.58,51.0,1.07,1.65,0.39,26.98,27.37,7.37,--,7.02,15.18,7.34,--,6.99,15.13,0.03,--,0.03,0.05,0.01,0.00,9.90,9.90,0.01,0.10,0.11,0.35,0.00,0.35,--,0.00,0.25,125.81,0.0,0.00,0.00,--,--,0.00,0.00,0.35,0.0,0.35,--,0.00,0.25,125.81,6.88,40.87,47.75,7.27,38083.0,6.65,2.39,-2.55,6.35,6.24,6.20,0.0,0.0,0.0,0.0,0.0,0.0,86.78,13.22,0.00,0.0,0.0,0.0
1,GA,McIntosh,13191,-81.2646,31.5329,13.969,10.497,0.000,10.497,1.04,0.00,1.04,3.472,75.0,0.94,90.0,0.26,1.20,0.02,0.00,0.02,0.16,--,0.16,0.15,0.00,--,0.00,0.00,0.16,--,0.16,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,--,--,0.00,0.00,0.0,0.00,0.00,--,--,0.00,0.00,0.00,0.0,0.00,--,--,0.00,0.00,1.48,0.00,1.48,0.16,42988.0,20.64,16.44,14.46,21.35,26.32,19.77,0.0,0.0,0.0,0.0,0.0,0.0,75.07,24.93,2.25,0.0,0.0,0.0
2,GA,Liberty,13179,-81.2103,31.7093,62.467,51.264,0.000,51.264,6.42,0.00,6.42,11.203,75.0,3.18,62.0,0.84,4.02,10.43,0.63,11.06,0.00,--,0.00,0.00,0.00,--,0.00,0.00,0.00,--,0.00,0.00,0.02,0.00,0.00,0.00,0.03,0.09,0.12,0.00,0.00,0.00,--,--,0.00,0.00,0.0,0.00,0.00,--,--,0.00,0.00,0.00,0.0,0.00,--,--,0.00,0.00,17.72,0.74,18.46,0.00,42201.0,20.14,15.37,14.95,20.68,25.22,19.19,0.0,0.0,0.0,0.0,0.0,0.0,84.02,15.98,0.21,0.0,0.0,0.0
86,CT,Hartford,9003,-72.7328,41.8064,895.841,78.563,671.826,750.389,12.78,46.44,59.22,145.452,34.0,25.29,34.0,4.90,30.19,9.02,97.25,106.27,4.09,--,3.90,7.09,2.41,--,2.30,4.95,1.68,--,1.60,2.14,0.07,0.79,12.96,13.75,0.11,0.48,0.59,0.00,71.56,71.56,--,0.43,0.98,614.78,0.0,71.56,71.56,--,--,0.67,421.59,0.00,0.0,0.00,--,0.43,0.31,193.19,28.24,232.21,260.45,4.88,66395.0,10.22,4.46,3.57,9.94,10.56,9.30,0.0,0.0,0.0,0.0,0.0,0.0,39.70,60.30,30.55,0.0,0.0,0.0
87,CT,Tolland,9013,-72.3365,41.8550,151.420,45.125,46.224,91.349,3.29,1.64,4.93,60.071,42.0,3.83,42.0,2.52,6.35,0.47,0.53,1.00,0.50,--,0.47,0.80,0.23,--,0.21,0.45,0.27,--,0.26,0.35,0.18,0.00,0.00,0.00,0.09,0.45,0.54,0.00,0.00,0.00,--,--,0.00,0.00,0.0,0.00,0.00,--,--,0.00,0.00,0.00,0.0,0.00,--,--,0.00,0.00,6.83,2.84,9.67,0.47,79626.0,9.85,4.09,2.84,9.55,10.03,8.88,0.0,0.0,0.0,0.0,0.0,0.0,40.99,59.01,29.40,0.0,0.0,0.0


# Replacing '--' to 0

In [ ]:
df.replace('--', 0, inplace=True)

In [ ]:
df.shape

(3196, 84)

In [ ]:
df.to_csv('../../data/clean-data/combined.csv', index=False)